In [2]:
from __future__ import annotations
import re
from dataclasses import dataclass, field
from collections import Counter
from typing import TypeVar, Dict, Set, Optional, Tuple
from copy import deepcopy
from config import DATA, MIN_SUP


# Define some type.
Item                        = TypeVar(int)
ItemList                    = TypeVar(list)
TID                         = TypeVar(int)
t_dict: Dict[TID, ItemList] = dict()
i_set:  Set[Item]           = set()
i_list: ItemList            = list()


# Formated data.
for line in open(DATA.test_sm):
    _ , _, tid, item, _ = re.split('[ \t\n]+', line)
    if tid in t_dict:
        t_dict[tid].append(item)
    else:
        t_dict[tid] = [item]
    i_set.add(item)
    i_list.append(item)

detail: bool = True

# Sort and filter out support value > min support.
counter = Counter(i_list)
if detail: print('='*100, f"\nCounter Dictionary:\n\t{dict(deepcopy(counter))}")

drop_out_items = {x[0] for x in filter(lambda i: i[1] < MIN_SUP, counter.items())}
if detail: print(f"drop_out_items:\n\t{deepcopy(drop_out_items)}")

after_filter = filter(lambda i: i[1] >= MIN_SUP, counter.items())
if detail: print(f"after_filter:\n\t{dict(deepcopy(after_filter))}")

after_sorted = sorted(after_filter, key=lambda i: i[1], reverse=True)
if detail: print(f"after_sorted:\n\t{dict(deepcopy(after_sorted))}\n", '='*100)


# Remove and order the transactions.
for k, v in deepcopy(t_dict).items():
    if detail: print(f"{k} before drop out:\n\t{list(deepcopy(v))}")
    after_droped_out = list(filter(lambda x: x not in drop_out_items, v))
    if detail: print(f"{k} after drop out:\n\t{list(deepcopy(after_droped_out))}")
    after_ordered = sorted(after_droped_out, key=lambda i: counter[i], reverse=True)
    if detail: print(f"{k} after ordered:\n\t{list(deepcopy(after_ordered))}\n", '='*100)
    t_dict[k] = after_ordered

Counter Dictionary:
	{'r': 3, 'z': 5, 'h': 1, 'j': 1, 'p': 2, 'y': 3, 'x': 4, 'w': 1, 'v': 1, 'u': 1, 't': 3, 's': 3, 'n': 1, 'o': 1, 'q': 2, 'e': 1, 'm': 1}
drop_out_items:
	{'n', 'p', 'q', 'e', 'w', 'h', 'o', 'm', 'v', 'j', 'u'}
after_filter:
	{'r': 3, 'z': 5, 'y': 3, 'x': 4, 't': 3, 's': 3}
after_sorted:
	{'z': 5, 'x': 4, 'r': 3, 'y': 3, 't': 3, 's': 3}
1 before drop out:
	['r', 'z', 'h', 'j', 'p']
1 after drop out:
	['r', 'z']
1 after ordered:
	['z', 'r']
2 before drop out:
	['z', 'y', 'x', 'w', 'v', 'u', 't', 's']
2 after drop out:
	['z', 'y', 'x', 't', 's']
2 after ordered:
	['z', 'x', 'y', 't', 's']
3 before drop out:
	['z']
3 after drop out:
	['z']
3 after ordered:
	['z']
4 before drop out:
	['r', 'x', 'n', 'o', 's']
4 after drop out:
	['r', 'x', 's']
4 after ordered:
	['x', 'r', 's']
5 before drop out:
	['y', 'r', 'x', 'z', 'q', 't', 'p']
5 after drop out:
	['y', 'r', 'x', 'z', 't']
5 after ordered:
	['z', 'x', 'y', 'r', 't']
6 before drop out:
	['y', 'z', 'x', 'e', 'q', 't', 

In [4]:
p_dict: Dict[TID, Tree] = dict()        # tree path dictionary

# Build FP-Tree
@dataclass
class Tree:
    tag: str
    val: Optional[int] = 1
    parent: Optional[Tree] = None
    children: Optional[Dict[str, Tree]] = field(default_factory=dict)

    def __getitem__(self, tag):
        return self.children[tag]

    def add(self, *tags: Tuple[str]):
        if not tags: return
        # print(self)
        first, other = tags[0], tags[1:]
        child = self.children.get(first, None)

        # First is a child.
        if child:
            child.val += 1
            child.add(*other)
        else:
            subtree = self.children[first] = Tree(first, parent=self)
            subtree.add(*other)

    def __str__(self, level=0):
        ret = f"{'  '*level + repr(self)}\n"
        for child in self.children.values():
            ret += child.__str__(level+1)
        return ret

    def __repr__(self):
        return f"<Tree {self.tag}: {self.val}>"

    def truncate(self):
        del self.children
        setattr(self, 'children', dict())

    def get_ancestors(self):
        parent = self.parent
        ancestors = []
        while parent:
            ancestors.append(parent.tag)
            parent = parent.parent
        return ancestors[::-1]

root = Tree('-', -1)
for k, v in deepcopy(t_dict).items():
    root.add(*v)

def build_p_dict(root):
    for child in root.children.values():
        if child.tag in p_dict:
            p_dict[child.tag].append(child.get_ancestors())
        else:
            p_dict[child.tag] = [child.get_ancestors()]
        build_p_dict(child)

build_p_dict(root)
p_dict


{'z': [['-']],
 'r': [['-', 'z'], ['-', 'z', 'x', 'y'], ['-', 'x']],
 'x': [['-', 'z'], ['-']],
 'y': [['-', 'z', 'x']],
 't': [['-', 'z', 'x', 'y'], ['-', 'z', 'x', 'y', 'r']],
 's': [['-', 'z', 'x', 'y', 't'], ['-', 'x', 'r']]}